### Generating a toy-case topology

In [32]:
import numpy as np 
import cvxpy as cp

from itertools import combinations
from tqdm import tqdm

In [33]:
# Let's generate a toy topology for our example

nodes = [i for i in range(7)]
edges = [
    (0,1),
    (0,3),
    (0,6),
    (1,2),
    (1,5),
    (2,4),
    (4,6),
    (5,6)
]

V = 7
E = len(edges)

d = 3                                           # Node and edges stalks dimension

F = {
    e:{
        e[0]:np.random.randn(d,d),
        e[1]:np.random.randn(d,d)
        } 
        for e in edges
    }                                           # Incidency linear maps

# Sheaf representation 

# Coboundary map

B = np.zeros((d*E, d*V))

for i in range(len(edges)):
    edge = edges[i]

    u = edge[0] 
    v = edge[1] 

    B_u = F[edge][u]
    B_v = F[edge][v]

    B[i*d:(i+1)*d, u*d:(u+1)*d] = B_u
    B[i*d:(i+1)*d, v*d:(v+1)*d] = - B_v

# Sheaf Laplacian

L_f = B.T @ B

# Generating a smooth signals dataset 

*(from Hansen J., "Learning sheaf Laplacians from smooth signals")* 

In order to retrieve a dataset of smoothsignals, first of all we sample random gaussians vectors on the nodes of the graph. Then we smooth them according to their expansion in terms of the eigenvectors of the sheaf Laplacian $L_0$.

So let's firstly define a dataset of random gaussian vectors. 

In [34]:
N = 100
X = np.random.randn(V*d,N)

Now we'll use the Fourier-domain embedded in the Laplacian spectrum. 

We'll consider a Tikhonov inspired procedure where we firstly project our dataset over the space spanned by the eigenvectors of the sheaf laplacian: namely $U$ the matrix collecting this eigenvectors we have 
\begin{equation}
    \hat{x} = U^T x
\end{equation}

So that defining $h(\lambda) = \frac{1}{1 + 10\lambda}$ and $H = \mathrm{diag}\{h(\lambda)\}_{\lambda}$, we now have

\begin{equation}
    \hat{y} = H(\Lambda) \hat{x}
\end{equation}

and finally our dataset is just reprojected back into the vertex domain:

\begin{equation}
    y = U H(\Lambda) \hat{x} = U H(\Lambda) U^T x
\end{equation}

In [35]:
Lambda, U = np.linalg.eig(L_f)
H = 1/(1 + 10*Lambda)

In [36]:
Y = U @ np.diag(H) @ U.T @ X

Y += np.random.normal(0, 10e-2, size=Y.shape)

In [37]:
np.trace(X.T @ L_f @ X)

11994.448108690382

In [38]:
np.trace(Y.T @ L_f @ Y)

130.2116372229304

____________

In [39]:
def solver(Y, d, edges, N):

    obj = 0
    trace = 0

    Fs = {
        edge: {
            edge[0]:None,
            edge[1]:None
        }
        for edge in edges
    }

    # Loop over edges to define variables and construct objective
    for edge in edges:
        u = edge[0]
        v = edge[1]

        Y_u = Y[u*d:(u+1)*d, :]
        Y_v = Y[v*d:(v+1)*d, :]

        # Define optimization variables for each edge
        Fs[edge][u] = cp.Variable((d, d))
        Fs[edge][v] = cp.Variable((d, d))

        # Update objective and trace
        obj += cp.norm(Fs[edge][u] @ Y_u - Fs[edge][v] @ Y_v, 'fro')**2
        trace += cp.trace(Fs[edge][u]) + cp.trace(Fs[edge][v])

    # Define the trace constraint
    trace_constraint = (trace == N)

    # Define the problem
    problem = cp.Problem(cp.Minimize(obj), [trace_constraint])

    # Solve the problem
    problem.solve(solver=cp.MOSEK,
                  mosek_params = { 'MSK_IPAR_INTPNT_SOLVE_FORM': 'MSK_SOLVE_PRIMAL' },
                  verbose=True)

    # Return the solution
    return Fs

In [40]:
Fs = solver(Y, d, edges,70)
B_hat = np.zeros((d*E, d*V))

for i in range(len(edges)):
    edge = edges[i]

    u = edge[0] 
    v = edge[1] 

    B_u = Fs[edge][u].value
    B_v = Fs[edge][v].value

    B_hat[i*d:(i+1)*d, u*d:(u+1)*d] = B_u
    B_hat[i*d:(i+1)*d, v*d:(v+1)*d] = - B_v

# Sheaf Laplacian

L_f_hat = B_hat.T @ B_hat
print(f'Average reconstruction error for trace barrier with N = {N}: {np.linalg.norm(L_f - L_f_hat) / L_f.size}')

                                     CVXPY                                     
                                     v1.4.2                                    
(CVXPY) Aug 20 03:26:00 PM: Your problem has 144 variables, 1 constraints, and 0 parameters.
(CVXPY) Aug 20 03:26:00 PM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Aug 20 03:26:00 PM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Aug 20 03:26:00 PM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
(CVXPY) Aug 20 03:26:00 PM: Your problem is compiled with the CPP canonicalization backend.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) Aug 20 03:26:00 PM: Compiling problem (target solver=SCS).
(CVX

In [41]:
np.trace(Y.T @ L_f @ Y)

130.2116372229304

In [42]:
np.trace(Y.T @ L_f_hat @ Y)

136.7388231035773

_______________